In [1]:
import sys
#print(sys.path)
#sys.path.append('C:\\Users\\user\\Anaconda3\\lib\\site-packages\\theano_bpr')
sys.path.append('C:\\Users\\Вадим\\Anaconda3\\lib\\site-packages\\theano_bpr')    
#print(sys.path)
from theano_bpr import BPR
import urllib
import urllib.request
import pandas as pd

In [2]:
def load_data_from_movielens(url, threshold, users_to_i = {}, items_to_i = {}):
    raw_data = []
    rat_data = []
    for line in urllib.request.urlopen(url).readlines():
        #line = [x.decode('utf8').strip() for x in z]
        user, item, rating, timestamp = line.decode().split('\t')
        if int(rating) > threshold:
            raw_data.append((user, item))
            rat_data.append((user, item, rating))
    return load_data_from_array(raw_data, rat_data)

In [3]:
def load_data_from_array(array, rat, users_to_i = {}, items_to_i = {}):
    data = []
    if len(users_to_i.values()) > 0:
        u = max(users_to_i.values()) + 1
    else:
        u = 0
    if len(items_to_i.values()) > 0:
        i = max(items_to_i.values()) + 1
    else:
        i = 0
    for user, item in array:
        if user not in users_to_i:
            users_to_i[user] = u
            u += 1
        if item not in items_to_i:
            items_to_i[item] = i
            i += 1
        data.append((users_to_i[user], items_to_i[item]))
    return data, users_to_i, items_to_i, rat

In [4]:
def train(self, train_data, epochs=30, batch_size=1000):
        """
          Trains the BPR Matrix Factorisation model using Stochastic
          Gradient Descent and minibatches over `train_data`.

          `train_data` is an array of (user_index, item_index) tuples.

          We first create a set of random samples from `train_data` for 
          training, of size `epochs` * size of `train_data`.

          We then iterate through the resulting training samples by
          batches of length `batch_size`, and run one iteration of gradient
          descent for the batch.
        """
        if len(train_data) < batch_size:
            sys.stderr.write("WARNING: Batch size is greater than number of training samples, switching to a batch size of %s\n" % str(len(train_data)))
            batch_size = len(train_data)
        _train_dict, _train_users, _train_items = _data_to_dict(train_data)
        n_sgd_samples = len(train_data) * epochs
        sgd_users, sgd_pos_items, sgd_neg_items = self._uniform_user_sampling(n_sgd_samples)
        z = 0
        t2 = t1 = t0 = time.time()
        while (z+1)*batch_size < n_sgd_samples:
            self.train_model(
                sgd_users[z*batch_size: (z+1)*batch_size],
                sgd_pos_items[z*batch_size: (z+1)*batch_size],
                sgd_neg_items[z*batch_size: (z+1)*batch_size]
            )
            z += 1
            t2 = time.time()
            sys.stderr.write("\rProcessed %s ( %.2f%% ) in %.4f seconds" %(str(z*batch_size), 100.0 * float(z*batch_size)/n_sgd_samples, t2 - t1))
            sys.stderr.flush()
            t1 = t2
        if n_sgd_samples > 0:
            sys.stderr.write("\nTotal training time %.2f seconds; %e per sample\n" % (t2 - t0, (t2 - t0)/n_sgd_samples))
            sys.stderr.flush()

In [5]:
training_data, users_to_index, items_to_index, raiting = load_data_from_movielens('http://files.grouplens.org/datasets/movielens/ml-100k/ua.base', 0)

In [6]:
testing_data, users_to_index, items_to_index, rt = load_data_from_movielens('http://files.grouplens.org/datasets/movielens/ml-100k/ua.test', 0, users_to_index, items_to_index)

In [8]:
testing_data

[(0, 914),
 (0, 665),
 (0, 731),
 (0, 427),
 (0, 666),
 (0, 667),
 (0, 510),
 (0, 335),
 (0, 428),
 (0, 511),
 (1, 12),
 (1, 47),
 (1, 241),
 (1, 732),
 (1, 512),
 (1, 689),
 (1, 733),
 (1, 429),
 (1, 690),
 (1, 734),
 (2, 235),
 (2, 279),
 (2, 735),
 (2, 330),
 (2, 736),
 (2, 691),
 (2, 513),
 (2, 1261),
 (2, 1024),
 (2, 737),
 (3, 47),
 (3, 250),
 (3, 254),
 (3, 275),
 (3, 279),
 (3, 287),
 (3, 328),
 (3, 514),
 (3, 430),
 (3, 1423),
 (4, 0),
 (4, 1),
 (4, 16),
 (4, 94),
 (4, 106),
 (4, 215),
 (4, 738),
 (4, 739),
 (4, 740),
 (4, 1177),
 (5, 13),
 (5, 21),
 (5, 65),
 (5, 82),
 (5, 94),
 (5, 248),
 (5, 285),
 (5, 515),
 (5, 741),
 (5, 692),
 (6, 30),
 (6, 156),
 (6, 354),
 (6, 401),
 (6, 424),
 (6, 451),
 (6, 741),
 (6, 468),
 (6, 984),
 (6, 742),
 (7, 20),
 (7, 47),
 (7, 75),
 (7, 85),
 (7, 174),
 (7, 279),
 (7, 313),
 (7, 357),
 (7, 426),
 (7, 526),
 (8, 5),
 (8, 273),
 (8, 282),
 (8, 315),
 (8, 451),
 (8, 459),
 (8, 478),
 (8, 491),
 (8, 497),
 (8, 743),
 (9, 6),
 (9, 15),
 (9, 96)

In [17]:
bpr = BPR(5, len(users_to_index.keys()), len(items_to_index.keys()))

In [18]:
bpr.train(training_data, epochs=50)

Generating 4528500 random training samples
Processed 4528000 ( 99.99% ) in 0.3710 seconds
Total training time 1744.69 seconds; 3.852681e-04 per sample


In [19]:
bpr.test(testing_data)

Current AUC mean (900 samples): 0.92707


0.92712472229941756

In [21]:
bpr = BPR(10, len(users_to_index.keys()), len(items_to_index.keys()))

In [22]:
bpr.train(training_data, epochs=50)

Generating 4528500 random training samples
Processed 4528000 ( 99.99% ) in 0.6030 seconds
Total training time 2718.36 seconds; 6.002775e-04 per sample


In [23]:
bpr.test(testing_data)

Current AUC mean (900 samples): 0.93095


0.93095685790917848

In [8]:
bpr = BPR(20, len(users_to_index.keys()), len(items_to_index.keys()))

ERROR (theano.gof.opt): Optimization failure due to: constant_folding
ERROR (theano.gof.opt): node: InplaceDimShuffle{x}(TensorConstant{0.0})
ERROR (theano.gof.opt): TRACEBACK:
ERROR (theano.gof.opt): Traceback (most recent call last):
  File "C:\Users\Вадим\Anaconda3\lib\site-packages\theano\gof\opt.py", line 1982, in process_node
    replacements = lopt.transform(node)
  File "C:\Users\Вадим\Anaconda3\lib\site-packages\theano\tensor\opt.py", line 6450, in constant_folding
    no_recycling=[], impl=impl)
  File "C:\Users\Вадим\Anaconda3\lib\site-packages\theano\gof\op.py", line 935, in make_thunk
    no_recycling)
  File "C:\Users\Вадим\Anaconda3\lib\site-packages\theano\gof\op.py", line 839, in make_c_thunk
    output_storage=node_output_storage)
  File "C:\Users\Вадим\Anaconda3\lib\site-packages\theano\gof\cc.py", line 1190, in make_thunk
    keep_lock=keep_lock)
  File "C:\Users\Вадим\Anaconda3\lib\site-packages\theano\gof\cc.py", line 1131, in __compile__
    keep_lock=keep_lock)


In [9]:
bpr.train(training_data, epochs=50)

Generating 4528500 random training samples
Processed 4528000 ( 99.99% ) in 0.0160 seconds
Total training time 75.88 seconds; 1.675706e-05 per sample


In [10]:
bpr.test(testing_data)

Current AUC mean (900 samples): 0.93078


0.93063788743803733

In [11]:
bpr = BPR(30, len(users_to_index.keys()), len(items_to_index.keys()))
bpr.train(training_data, epochs=50)
bpr.test(testing_data)

Generating 4528500 random training samples
Processed 4528000 ( 99.99% ) in 0.0180 seconds
Total training time 84.24 seconds; 1.860193e-05 per sample
Current AUC mean (900 samples): 0.92835


0.92846303077698378

In [12]:
bpr = BPR(50, len(users_to_index.keys()), len(items_to_index.keys()))
bpr.train(training_data, epochs=50)
bpr.test(testing_data)

Generating 4528500 random training samples
Processed 4528000 ( 99.99% ) in 0.0220 seconds
Total training time 98.56 seconds; 2.176342e-05 per sample
Current AUC mean (900 samples): 0.92745


0.92760479495623449

In [13]:
bpr = BPR(100, len(users_to_index.keys()), len(items_to_index.keys()))
bpr.train(training_data, epochs=50)
bpr.test(testing_data)

Generating 4528500 random training samples
Processed 4528000 ( 99.99% ) in 0.0280 seconds
Total training time 138.33 seconds; 3.054740e-05 per sample
Current AUC mean (900 samples): 0.92688


0.92683406253237222

In [9]:
aucsbpr=[0.92712472229941756, 
         0.93095685790917848, 
         0.93063788743803733,
         0.92846303077698378,
         0.92760479495623449,
         0.92683406253237222]

In [10]:
aucsbpr

[0.9271247222994176,
 0.9309568579091785,
 0.9306378874380373,
 0.9284630307769838,
 0.9276047949562345,
 0.9268340625323722]